In [ ]:
## This file is just a way to visualize any search grids for the main script. It is not required to run the main script.
import folium
import math


def haversine_distance(lat1, lon1, lat2, lon2):
  lat1, lon1, lat2, lon2 = map(math.radians, [lat1, lon1, lat2, lon2])

  dlon = lon2 - lon1
  dlat = lat2 - lat1
  a = math.sin(dlat / 2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2)**2
  c = 2 * math.asin(math.sqrt(a))
  r = 6378 #radius of earth in km
  distance = c * r

  return distance





def get_start_end_read_list():
    startlat = None
    startlong = None
    endlat = None
    endlong = None

    while startlat is None:
      try:
        startlat = float(input('Where does the journey start? (latitude) '))
        if not (-90 <= startlat <= 90):
          print('Invalid input. Please enter valid coordinates.')
          startlat = None
      except ValueError:
        print('Invalid input. Please enter numeric coordinates.')

    while startlong is None:
      try:
        startlong = float(input('Where does the journey start? (longitude) '))
        if not (-180 <= startlong <= 180):
          print('Invalid input. Please enter valid coordinates.')
          startlong = None
      except ValueError:
        print('Invalid input. Please enter numeric coordinates.')

    while endlat is None:
      try:
        endlat = float(input('Where does the journey end? (latitude) '))
        if not (-90 <= endlat <= 90):
          print('Invalid input. Please enter valid coordinates.')
          endlat = None
      except ValueError:
          print('Invalid input. Please enter numeric coordinates.')

    while endlong is None:
      try:
        endlong = float(input('Where does the journey end? (longitude) '))
        if not (-180 <= endlong <= 180):
          print('Invalid input. Please enter valid coordinates.')
          endlong = None
      except ValueError:
        print('Invalid input. Please enter numeric coordinates.')

    print(f'Journey distance: {haversine_distance(startlat, startlong, endlat, endlong)} km')
    return startlat, startlong, endlat, endlong

startlat, startlong, endlat, endlong = get_start_end_read_list()




def get_main_coordinates(startlat, startlong, endlat, endlong, r):
  interval = math.sqrt(2)*r
  coordlist = []
  earth_radius = 6378  #Earth radius in km
  distance = haversine_distance(startlat, startlong, endlat, endlong)
  N = distance/interval
  if N < 1:
    N = 1

  startlat_rad = math.radians(startlat)
  startlong_rad = math.radians(startlong)
  endlat_rad = math.radians(endlat)
  endlong_rad = math.radians(endlong)

  d_lat_rad = (endlat_rad - startlat_rad)/N
  d_long_rad = (endlong_rad - startlong_rad)/N

  first_lat_rad = startlat_rad + (endlat_rad - startlat_rad)/(2*N)
  first_long_rad = startlong_rad + (endlong_rad - startlong_rad)/(2*N)
  first_lat = math.degrees(first_lat_rad)
  first_long = math.degrees(first_long_rad)
  coordlist.append((first_lat, first_long))
  currentcoord = (first_lat_rad, first_long_rad)

  for i in range(abs(int(N-1))):
    new_lat_rad = currentcoord[0] + d_lat_rad
    new_long_rad = currentcoord[1] + d_long_rad
    coordlist.append((math.degrees(new_lat_rad), math.degrees(new_long_rad)))
    currentcoord = (new_lat_rad, new_long_rad)

  return coordlist

main_coords = get_main_coordinates(startlat, startlong, endlat, endlong, 10)




def get_sub_coordinates(mainlat, mainlong, prevlat, prevlong, r, n):
  interval = math.sqrt(2)*r
  subcoords = []
  earth_radius = 6378  #Earth radius in km

  mainlat_rad = math.radians(mainlat)
  mainlong_rad = math.radians(mainlong)
  prevlat_rad = math.radians(prevlat)
  prevlong_rad = math.radians(prevlong)

  subcoords.append((mainlat, mainlong))

  delta_long = mainlong_rad - prevlong_rad
  alpha = math.atan2(math.sin(delta_long) * math.cos(mainlat_rad),
    math.cos(prevlat_rad) * math.sin(mainlat_rad)- math.sin(prevlat_rad) * math.cos(mainlat_rad) * math.cos(delta_long))

  beta = (alpha + math.pi/2, alpha - math.pi/2)

  for angle in beta:
    currentcoord = (mainlat_rad, mainlong_rad)
    for i in range(n):
      sublat_rad = currentcoord[0] + (interval/earth_radius) * math.cos(angle)
      sublong_rad = currentcoord[1] + (interval/earth_radius) * math.sin(angle)/math.cos(mainlat_rad)
      sublat = math.degrees(sublat_rad)
      sublong = math.degrees(sublong_rad)
      subcoords.append((sublat, sublong))
      currentcoord = (sublat_rad, sublong_rad)

  return subcoords

subcoords=[]
for i in range(len(main_coords)):
  n = (0.08*(len(main_coords)+1)*(1-math.cos((2*math.pi*(i+1))/(len(main_coords)+1))))  #n is the number of circles we want to search above the main coordinates. This nuber is higher towards the middle of the route and lower towards the ends.
  #print(i, n, round(n))
  column = get_sub_coordinates(main_coords[i][0], main_coords[i][1], main_coords[i-1][0], main_coords[i-1][1], 10, round(n))
  #print(column)
  for coord in column:
    subcoords.append(coord)
number_of_coords = len(subcoords)




# map
center_lat = sum(coord[0] for coord in main_coords) / len(main_coords)
center_lon = sum(coord[1] for coord in main_coords) / len(main_coords)

the_map = folium.Map(location=[center_lat, center_lon], zoom_start=9)

folium.Marker(location=[38.25028316852972, -96.22721594417085]).add_to(the_map)
folium.Marker(location=[36.510983856987394, -94.27925413984879]).add_to(the_map)

folium.Circle(location=[37.136328283088446, -94.55254010322729], radius=10000, color='red', fill=True).add_to(the_map)

for coord in main_coords:
  folium.Marker(location=coord).add_to(the_map)
  folium.Circle(location=coord, radius=10000, color='red', fill=False).add_to(the_map)

for coords in subcoords:
  folium.Marker(location=coords).add_to(the_map)
  folium.Circle(location=coords, radius=10000, color='blue', fill=False).add_to(the_map)

display(the_map);

Journey distance: 279.76329683672816 km
